[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DSIMB/PoincareMSA/blob/master/PoincareMSA_colab.ipynb)

<img src="https://github.com/DSIMB/PoincareMSA/blob/master/.github/PoincareMSA_small_logo.png?raw=true" height="100" style="height:100px;margin-left: 0px;">

# Poincaré maps for visualization of large protein famillies

**Authors**: Anna Klimovskaia Susmelj, Yani Ren, Yann Vander Meersche, Jean-Christophe Gelly and Tatiana Galochkina

PoincaréMSA builds an interactive projection of an input protein multiple sequence alignemnt (MSA) using a method based on Poincaré maps described by Klimovskaia et al [1]. It reproduces both local proximities of protein sequences and hierarchy contained in give data. Thus, sequences located closer to the center of projection correspond to the proteins sharing the most general functional properites and/or appearing at the earlier stages of evolution. Source code is available at https://github.com/DSIMB/PoincareMSA.

[1] Klimovskaia, A., Lopez-Paz, D., Bottou, L. et al. Poincaré maps for analyzing complex hierarchies in single-cell data. Nat Commun 11, 2966 (2020).

# Directory Parameters

In [70]:
# Update working directory

%cd /home/hugo/Bureau/PoincareMSA/

/home/hugo/Bureau/PoincareMSA


In [71]:
import sys

# Add the project root to Python path
project_root = "/home/hugo/Bureau/PoincareMSA"
if project_root not in sys.path:
    sys.path.append(project_root)

# Parameters

In [72]:
# GENERAL OPTIONS ==========================================
# Embedding or without
data_type = "pssm" #  "pssm"   "plm"   "RFA_matrix"   "plm_aae"   "distance_matrix"
#----------------------------------------------------------
#Annotation file (.csv) or UniProt ID list. (Emtpy strings for no annotations)
path_annotation_csv = "examples/globins/globin_colors_new.csv"   # Path or ""
# OR
path_uniprot_list = ""   # Path or ""
#==========================================================

# POINCARE PARAMETERS ====================================
# Here you control different parameters of Poincaré maps.
# In our computational experiments the best results were achieved for the following values provided by default.
# The impact of different parameters is analyzed in the original paper [1].
knn = 5
gamma = 1
sigma = 1
cospca = 0
batchs = 4
epochs = 10
seed = 4
distance = "minkowski"  #"minkowski"  "cosine"
#==========================================================

In [73]:
# OPTIONS WITH EMBEDDING ===============================
if data_type == "plm" :
    # Input the embedding folder here
    embedding_path = 'embeddings/ankh_base_globins/'
    #--------------------------------------------------
    in_name = 'embeddings/ankh_base_globins/'  # Input here the name of folder with the embeddings
    mid_output = 'globins_data/with_plm_embeddings/'  # Input name of desired folder for intermediary results
    out_name_results = 'results/globins/with_plm_embeddings/' # Input desired name of output folder
#==========================================================

In [74]:
# OPTIONS WITH EMBEDDING AUTO ENCODER ==================
if data_type == "plm_aae" :
    # Input the embedding folder here
    embedding_path = 'embeddings/aae_embeddings/ankh_base_globins/'
    #--------------------------------------------------
    in_name = 'embeddings/aae_embeddings/ankh_base_globins/'  # Input here the name of folder with the embeddings
    mid_output = 'globins_data/with_aae_plm_embeddings/'  # Input name of desired folder for intermediary results
    out_name_results = 'results/globins/with_aae_plm_embeddings/' # Input desired name of output folder
#==========================================================

In [75]:
# OPTIONS WITHOUT EMBEDDING ===============================
if data_type == "pssm" :
    # Input the mfasta file here
    mfasta = 'examples/globins/glob.mfasta'
    # Name for the output folder
    out_name = "globins_data"
    #----------------------------------------------------------
    # Threshold for filtering gapped positions
    # Positions with proportion of gaps above the given threshold are removed from the alignment.
    # If your alignment is very gapped, you may want to increase this value.
    gapth = 0.9 
    #----------------------------------------------------------
    in_name = 'examples/globins/globins_data/fasta0.9/'  # Input here the name of folder with the fasta files
    mid_output = 'globins_data/with_mfasta/'  # Input name of desired folder for intermediary results
    out_name_results = 'results/globins/with_mfasta/' # Input desired name of output folder
#==========================================================

In [76]:
# OPTIONS WITH RFA MATRIX ================================
if data_type == "RFA_matrix" :
    # Input the RFA matrix file here
    mid_output = 'RFA_matrix/globins/with_plm_embeddings/'
    # Input True if the matrix comes from plm and False for pssm
    matrix_plm = True
    # Folder for the results
    out_name_results = 'RFA_matrix/globins/with_plm_embeddings/results/' # Input desired name of output folder
#==========================================================

In [77]:
# OPTIONS WITH DISTANCE MATRIX ============================
if data_type == "distance_matrix" :
    # Input the distance matrix file here
    distance_matrix = 'distance_matrix/globins/with_mfasta/distance_matrix.csv'
    mid_output = 'distance_matrix/globins/with_mfasta/outputs/'
    labels = 'distance_matrix/globins/with_mfasta/labels.csv'
    # Input True if the matrix comes from plm and False for pssm
    matrix_plm = True
    # Folder for the results
    out_name_results = 'distance_matrix/globins/with_mfasta/results/' # Input desired name of output folder
#==========================================================

# Librairies

In [78]:
#Load dependencies
import os
import numpy as np
import pandas as pd
import subprocess
import json
import warnings
warnings.filterwarnings('ignore')

#Import visualization functions
from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding, plot_embedding_interactive, rotate, get_colors
from scripts.prepare_data.mmseqs2_api import run_mmseqs2
from scripts.prepare_data.uniprot_idmapping_api import submit_id_mapping, check_id_mapping_results_ready, get_id_mapping_results_link, get_id_mapping_results_search
%matplotlib inline

#Create optional variables
path_annotation = ""

# Data import

In [79]:
#Check files
#mfasta
nb_seq = 0

if data_type == "pssm" :
    if os.path.isfile(mfasta):
        with open(mfasta, "r") as f_in:
            for line in f_in:
                if line[0] == ">":
                    nb_seq += 1
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"File {mfasta} not found.")

elif data_type == "plm" or data_type == "plm_aae":
    if os.path.exists(embedding_path):
        nb_seq = len([s for s in os.listdir(embedding_path) if '.pt' in s])
        print(f"\nNumber of sequences found: {nb_seq}.")
    else:
        print(f"Folder {embedding_path} not found.")

elif data_type == "RFA_matrix" :
    print(f"RFA matrix used")

elif data_type == "Distance_matrix" :
    print(f"Distance matrix used")
    
else:
    print('Neither a valid mfasta or embedding folder has been provided')

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

#Check that only one path is selected
if path_annotation_csv and path_uniprot_list:
    raise ValueError("Use only one file path (path_annotation_csv OR path_uniprot_list).")

if path_annotation_csv :
    if os.path.isfile(path_annotation_csv):
        try:
            df_annotation = pd.read_csv(path_annotation_csv)
        except:
            raise ValueError("Annotation file is not in .csv format.")
        else:
            if len(df_annotation) != nb_seq and (data_type != "RFA_matrix" and data_type != "distance_matrix"):
                raise ValueError("Annotation file doesn't match the .mfasta file length.")

        #Add id column
        if "proteins_id" not in df_annotation.columns:
            df_annotation.insert(0, "proteins_id", range(len(df_annotation)))
        path_annotation = path_annotation_csv
            
        print("\nAnnotation file correctly loaded.")
        annotation_names = list(df_annotation.columns)
        print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_annotation_csv} not found.")

elif path_uniprot_list and (data_type != "RFA_matrix" and data_type != "distance_matrix"):
    if os.path.isfile(path_uniprot_list):
        try:
            UnP_ids = np.genfromtxt(path_uniprot_list, dtype="str")
        except:
            raise ValueError("UniProt IDs file is not in a valid format.")
        else:
            if len(UnP_ids) != nb_seq:
                raise ValueError("UniProt IDs file doesn't match the .mfasta file length.")     

            #Split UniProtKB and UniParc IDs
            uniparc_ids = []
            uniprot_ids = []
            for unp in UnP_ids:
                if len(unp) == 13 and unp[:2] == "UP":
                    uniparc_ids.append(unp)
                else:
                    uniprot_ids.append(unp)

            #Fetch UniProtKB annotations
            job_id = submit_id_mapping(
                from_db="UniProtKB_AC-ID", to_db="UniParc", ids=uniprot_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results = get_id_mapping_results_search(link)

            #Fetch UniParc annotations
            job_id = submit_id_mapping(
                from_db="UniParc", to_db="UniParc", ids=uniparc_ids
            )

            if check_id_mapping_results_ready(job_id):
                link = get_id_mapping_results_link(job_id)
                results2 = get_id_mapping_results_search(link)

            #Create annotation dataframe
            df_annotation = pd.DataFrame(UnP_ids, columns=["UnP_ID"])
            df_annotation[["organism", "proteinName", "taxonId", "species", "genus", \
                           "family", "order", "class", "phylum", "clade", "superkingdom"]] = ""

            #Fill the annotation DataFrame
            for dict_res in results["results"] + results2["results"]:
                try:
                    unp = dict_res["from"]
                    prot_name = dict_res["to"]["uniParcCrossReferences"][0]["proteinName"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "proteinName"] = prot_name
                    scientific_name = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["scientificName"]
                    taxid = dict_res["to"]["uniParcCrossReferences"][0]["organism"]["taxonId"]
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "organism"] = scientific_name
                    df_annotation.loc[df_annotation["UnP_ID"] == unp, "taxonId"] = taxid
                except KeyError:
                    continue

            #Add lineage from NCBI Taxonomist
            taxon_ids = df_annotation.loc[df_annotation["taxonId"].notnull(), 'taxonId'].to_numpy()
            taxon_ids = list(set(taxon_ids))
            taxon_ids = list(map(str, taxon_ids))
            bash_command = f"ncbi-taxonomist resolve -t {','.join(taxon_ids)}"
            list_taxon = subprocess.run(bash_command, shell=True, capture_output=True, text=True).stdout.strip().split("\n")

            if list_taxon != [""]:
                for taxon in list_taxon:
                    jsonString = taxon
                    taxon_dict = json.loads(jsonString)
                    query = taxon_dict["query"]
                    for lineage in taxon_dict["lineage"]:
                        rank = lineage["rank"]
                        if rank in ["species", "genus", "family", "order", "class", "phylum", "clade", "superkingdom"]:
                            name = lineage["name"]
                            df_annotation.loc[df_annotation["taxonId"] == int(query), rank] = name

            #Add id column
            if "proteins_id" not in df_annotation.columns:
                df_annotation.insert(0, "proteins_id", range(len(df_annotation)))

            #Save annotation to csv
            path_annotation = "auto_annot.csv"
            df_annotation.to_csv(path_annotation, index=False)

            print("\nAnnotation correctly fetched.")
            annotation_names = list(df_annotation.columns)
            print(f"{len(annotation_names)} annotations found: {annotation_names}.")
    else:
        print(f"File {path_uniprot_list} not found.")
elif data_type == "RFA_matrix":
    print(f"\nDistance matrix used")
    path_annotation = "auto_annot.csv"
else:
    print("No annotation file selected.")
    path_annotation = ""


Number of sequences found: 252.

Annotation file correctly loaded.
17 annotations found: ['proteins_id', 'tree1', 'tree2', 'tree3', 'tree4', 'full_name', 'short_name', 'full_species', 'short_species', 'evo_distance', 'Color_species', 'Domain', 'Kingdom', 'Phylum', 'Subphylum', 'Class', 'Genus'].


# Projection

## Function for bash building command

In [80]:
if data_type in ["RFA_matrix", "distance_matrix"]:
        in_name = "None"
        

def build_command(data_type=data_type, project_root=project_root, in_name=in_name, out_name_results=out_name_results, 
                  mid_output=mid_output, distance=distance, gamma=gamma, cospca=cospca, epochs=epochs, seed=seed, knn=knn):

    if data_type in ["plm", "plm_aae"]:
        plm_flag = "True"
    elif data_type == "pssm":
        plm_flag = "False"
    elif data_type in ["RFA_matrix", "distance_matrix"]:
        plm_flag = matrix_plm

    if data_type != "distance_matrix":
        cmd = (
            f"PYTHONPATH={project_root}:$PYTHONPATH python scripts/build_poincare_map/main.py "
            f"--input_path {in_name} "
            f"--output_path {out_name_results} "
            f"--plm_embedding {plm_flag} "
            f"--matrices_output_path {mid_output} "
            f"--distlocal {distance} "
            f"--gamma {gamma} "
            f"--pca {cospca} "
            f"--epochs {epochs} "
            f"--seed {seed} "
            f"--knn {knn} "
            f"--method {data_type} "
        )
        
    elif data_type == "distance_matrix":
        cmd = (
            f"PYTHONPATH={project_root}:$PYTHONPATH python scripts/build_poincare_map/main.py "
            f"--input_path {in_name} "
            f"--output_path {out_name_results} "
            f"--plm_embedding {plm_flag} "
            f"--matrices_output_path {mid_output} "
            f"--distlocal {distance} "
            f"--gamma {gamma} "
            f"--pca {cospca} "
            f"--epochs {epochs} "
            f"--seed {seed} "
            f"--knn {knn} "
            f"--method {data_type} "
            f"--distance_matrix {distance_matrix} "
            f"--labels {labels} "
        )

    return cmd

## Data preparation for pssm
Here we clean the input .mfasta alignment and translate each sequence to a vector ready for projection.

In [81]:
if data_type == "pssm" : 
    # Run data preparation
    #Data preparation consists in `.mfasta` cleaning according to a gap threshold and
    #translation of each sequence to the PSSM profile.
    prep_parameters = "scripts/prepare_data" + " " + mfasta + " " + out_name + " " + out_name + " " + str(gapth)
    bash_projection = "bash scripts/prepare_data/create_projection.sh " + prep_parameters
    !{bash_projection}

Input file: examples/globins/glob.mfasta
Name of the protein family: glob
80 X aa replaced by gaps in 252 sequences
filter_gaps finished for examples/globins/glob.mfasta
mfasta2fasta finished for globins_data/globins_data.clean0.9.mfasta


## Projection using bash command

In [ ]:
cmd = build_command()
print("CMD:", cmd)
os.system(cmd)

CMD: PYTHONPATH=/home/hugo/Bureau/PoincareMSA:$PYTHONPATH python scripts/build_poincare_map/main.py --input_path examples/globins/globins_data/fasta0.9/ --output_path results/globins/with_mfasta/ --plm_embedding False --matrices_output_path globins_data/with_mfasta/ --distlocal minkowski --gamma 1 --pca 0 --epochs 10 --seed 4 --knn 5 --method pssm 


INFO: CUDA available: True
INFO: Random seed set as 4
INFO: 252 proteins found in folder examples/globins/globins_data/fasta0.9/.
INFO: No root detected
INFO: Prepare data: tensor construction
INFO: Prepare data: successfully terminated
INFO: features CSV file saved to globins_data/with_mfasta/features.csv
INFO: labels CSV file saved to globins_data/with_mfasta/labels.csv
INFO: KNN matrix CSV file saved to globins_data/with_mfasta/KNN_matrix.csv
INFO: Computing laplacian...
INFO: Laplacian computed in 0.17 sec
INFO: Computing RFA...
INFO: RFA computed in 0.00 sec
INFO: RFA matrix computed (tensor shape (252, 252))
INFO: RFA matrix CSV file saved to globins_data/with_mfasta/RFA_matrix.csv
INFO: Starting training...
loss: 3.96986: 100%|████████████████████████████| 10/10 [00:01<00:00,  5.61it/s]


PM computed in 1.78 sec


INFO: 
loss = 3.970e+00
time = 0.038 min


0

# Projection visualization

## Preparation

In [ ]:
# Prepare data for visualization
#Check that an annotation file was provided. Create a dummy one instead
if not path_annotation:
    df_annotation = pd.DataFrame(list(zip(list(range(1,nb_seq+1)), np.full(nb_seq, "-", dtype=object))), columns=["id", "default"])
    df_annotation.to_csv("dummy_annotation.csv", index=False)
    path_annotation = "dummy_annotation.csv"
    annotation_names = ["proteins_id"]


path_embedding = f"{out_name_results}/PM{knn:1.0f}sigma={sigma:2.2f}gamma={gamma:2.2f}{distance}pca={cospca:1.0f}_seed{seed:1.0f}.csv"

df_embedding = read_embeddings(path_embedding, path_annotation, withroot=False)

#Here are different labels found in your annotation file (if one uploaded):
print(f"{len(annotation_names)} annotations found: {annotation_names}.")

17 annotations found: ['proteins_id', 'tree1', 'tree2', 'tree3', 'tree4', 'full_name', 'short_name', 'full_species', 'short_species', 'evo_distance', 'Color_species', 'Domain', 'Kingdom', 'Phylum', 'Subphylum', 'Class', 'Genus'].


### Create interactive plot

In [ ]:
# Construction of custom color palette 
globin_palette = {'Echinodermata': '#086b75', 'Arthropoda': '#0b237c', 'Mollusca': '#512ff8', 'Annelida': '#a191f3', 'Chordata': '#26c9d9', 'Cnidaria': '#ad288b', 'Porifera': '#fdb7fd', 'Placozoa': '#e9bd6b', 'Bacteria': '#f10000', 'Nematoda': '#5d78e3', 'Hemichordata': '#b0ffe8', 'Fungi': '#a0e361', 'Viridiplantae': '#4d9b03'}

## Visualization

In [ ]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:
if data_type == "plm" :
    title = "Poincaré projection of globins colored according family name with embeddings"  
elif data_type == "plm_aae" :
    title = "Poincaré projection of globins colored according family name with embeddings autoencoder" 
elif data_type == "pssm" :
    title = "Poincaré projection of globins colored according family name without embeddings" 
else :
    title = "Poincaré projection of globins colored according family name using a RFA matrix" 
#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "Color_species"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = globin_palette #Default: None
use_custom_palette = True
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

In [ ]:
# OPTIONS =================================================
#Here you can set different parameters to color & annotate the resulting projection:
if data_type == "plm" :
    title = "PM projection on globins colored by phylogenetic tree cluster with embeddings" 
elif data_type == "plm_aae" :
    title = "PM projection on globins colored by phylogenetic tree cluster with embeddings auto encoder" 
elif data_type == "pssm" :
    title = "PM projection on globins colored by phylogenetic tree cluster without embeddings" 
else :
    title = "PM projection on globins colored by phylogenetic tree cluster with RFA matrix" 

#----------------------------------------------------------
# Select the coloring from annotation .csv file:
labels_name = "tree3"
# Select classes to label among the "labels_name" or "second_labels_name" column (comma separated list):
second_labels_name = ""
labels_text = []
show_text = False
#----------------------------------------------------------
# Use a custom color palette:
color_palette = None #Default: None
use_custom_palette = False
#==========================================================


#Check projection visualization parameters
#Labels name
if labels_name == "":
    labels_name = None
elif labels_name not in annotation_names:
    raise NameError(f"labels_name {labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}")
#Second labels name
if second_labels_name == "":
    second_labels_name = None
elif second_labels_name not in annotation_names:
    raise NameError(f'"second_labels_name" {second_labels_name} is not in the availables annotations.\nAvailables annotations: {annotation_names}')

if not use_custom_palette:
    color_palette = None

#Plot graph
fig = plot_embedding_interactive(df_embedding, 
                                 labels_name = labels_name,
                                 second_labels_name = second_labels_name, 
                                 show_text = show_text,
                                 labels_text = labels_text,
                                 color_palette = color_palette, 
                                 title = title, 
                                 fontsize = 11)
fig.show()

### Save plot to file

In [ ]:
# OPTIONS =================================================
output_name = "globins_w_embeddings_by_family"
output_format = "html" #Format availables: ["png", "html", "pdf", "svg"]
#==========================================================


if output_format != "html":
    fig.write_image(f"{output_name}.{output_format}", engine="kaleido")
else:
    fig.write_html(f"{output_name}.{output_format}")